In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
# Read the dataset
data = pd.read_csv ('../input/competitive-data-science-predict-future-sales/sales_train.csv')
data.head(5)

In [ ]:
print("Count of items this sold",len(data))
data.describe()

In [ ]:
#Check for nan/nulls in the data
data.isnull().values.any()
print ('Count of NaN: ',data.isnull().sum().sum())


In [ ]:
pt = pd.pivot_table(data, index = ['shop_id', 'item_id'], values = 'item_cnt_day', columns = ['date_block_num'], aggfunc = np.sum, fill_value = 0)
pt.reset_index(inplace = True)
pt

In [ ]:
#Check for nan/nulls in the data
pt.isnull().values.any()
print ('Count of NaN: ',pt.isnull().sum().sum())

In [ ]:
X = pt.drop(columns=['shop_id','item_id', 33], axis=1)
y = pt[33]

In [ ]:
data_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
#now that we have item data, we merge the pivot table with the test data, given priority to the latter
df_test = pd.merge(data_test, pt, on=['shop_id', 'item_id'], how = 'left')
df_test.head(5)

In [ ]:
#as seen above we are missing values
df_test.fillna(0, inplace=True)
df_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [ ]:
X_Test = df_test.drop(columns=['shop_id','item_id','ID', 0], axis=1)
X_Test.columns = X_train.columns

In [ ]:
from xgboost import XGBRegressor
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# create an xgboost regression model
model = XGBRegressor(max_depth = 30,  n_estimators = 50)
# fit model
model.fit(X_train, y_train)

In [ ]:
# make a prediction
y_pred = model.predict(X_test)

In [ ]:
import sklearn
import math
mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print(rmse)

In [ ]:
yhat = model.predict(X_Test)

In [ ]:
submission = pd.DataFrame({
            "ID": np.arange(len(yhat)),
            "item_cnt_month": yhat
    })

In [ ]:
submission.to_csv('submission.csv', index=False)